https://www.the-numbers.com/movie/budgets/all
https://www.the-numbers.com/movie/budgets/all/5701

In [25]:
# imports
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import matplotlib.pyplot as plt 

In [26]:
# contants
MAX_PAGE = 5701 #last page number 
BASE_URL = "https://www.the-numbers.com/movie/budgets/all/"

In [27]:
# web scraping, API
page = '1'
respond = requests.get(BASE_URL + page)
respond.status_code

200

In [28]:
soup = BeautifulSoup(respond.text)

In [29]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
  </script>
  <meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
  <!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="telephone=no" name="format-detection"/>
  <!-- for apple mobile -->
  <meta content="521546213" property="fb:admins"/>
  <meta content="initial-scale=1" name="viewport"/>
  <meta content="The budge

In [30]:
tables = soup.find_all('table') # search tab <table>, return soup (list)
len(tables), type(tables)

(1, bs4.element.ResultSet)

In [31]:
tables[0]

<table>
<tr><th> </th><th>Release Date</th><th>Movie</th><th>Production Budget</th><th>Domestic Gross</th><th>Worldwide Gross</th></tr>
<tr><td class="data">1</td>
<td><a href="/box-office-chart/daily/2009/12/18">Dec 18, 2009</a></td>
<td><b><a href="/movie/Avatar#tab=summary">Avatar</a></b></td>
<td class="data">$425,000,000</td>
<td class="data">$760,507,625</td>
<td class="data">$2,776,345,279</td>
</tr>
<tr><td class="data">2</td>
<td><a href="/box-office-chart/daily/2011/05/20">May 20, 2011</a></td>
<td><b><a href="/movie/Pirates-of-the-Caribbean-On-Stranger-Tides#tab=summary">Pirates of the Caribbean: On Stranger Tides</a></b></td>
<td class="data">$410,600,000</td>
<td class="data">$241,063,875</td>
<td class="data">$1,045,663,875</td>
</tr>
<tr><td class="data">3</td>
<td><a href="/box-office-chart/daily/2019/06/07">Jun 7, 2019</a></td>
<td><b><a href="/movie/Dark-Phoenix-(2019)#tab=summary">Dark Phoenix</a></b></td>
<td class="data">$350,000,000</td>
<td class="data">$60,542,4

In [32]:
# <tr>  = table rows in html
# <td> = table data
# <th> = table header
# google html table tab , e.g, W3schools
table = tables[0]
type(table), len(table.find_all('tr'))

(bs4.element.Tag, 101)

In [33]:
len(table.find_all('th'))

6

In [34]:
for th in table.find_all('th'):
    print(th.text) # blank header for idx in the table 

 
Release Date
Movie
Production Budget
Domestic Gross
Worldwide Gross


In [35]:
headers = [th.text for th in table.find_all('th')] #[]list, ()tuple, {}dict comprehension
headers

['\xa0',
 'Release Date',
 'Movie',
 'Production Budget',
 'Domestic Gross',
 'Worldwide Gross']

In [36]:
headers[0] = 'Index'
headers

['Index',
 'Release Date',
 'Movie',
 'Production Budget',
 'Domestic Gross',
 'Worldwide Gross']

In [37]:
rows = table.find_all('tr') #find all the rows in the table 
data = []
# for i, row in enumerate(rows):
#for row in rows[1:0]:  # row 0 is the header
#    data.append((td.text for td in row.find_all('td'))) #tuple comprehension 

In [38]:
row = rows[10]
print(row)

<tr><td class="data">10</td>
<td><a href="/box-office-chart/daily/2015/11/06">Nov 6, 2015</a></td>
<td><b><a href="/movie/Spectre#tab=summary">Spectre</a></b></td>
<td class="data">$300,000,000</td>
<td class="data">$200,074,175</td>
<td class="data">$879,620,923</td>
</tr>


In [39]:
for row in rows[1:]:
    data_row = [] # create a new row for data table
    tds = row.find_all('td')
    # 1st element only has text
    data_row.append(tds[0].text) #find brings 1st 
    # 2nd element only has text
    data_row.append(tds[1].find('a').text)
    # grad link in 2nd element 
    data_row.append(tds[1].find('a')['href'])
    # 3rd elem & link
    data_row.append(tds[2].find('a').text)
    data_row.append(tds[2].find('a')['href'])
    # 4th-6th only has text
    data_row.append(tds[3].text)
    data_row.append(tds[4].text)
    data_row.append(tds[5].text)
    # print * = unpack
#    print(*data_row, sep='\n')
    data.append(data_row) # add rows to data table 
    
print(len(data))    

100


## wrap above to a function

In [40]:
def process_page(soup):
    '''
    Take a page from request,
    Outputs a list of lists (table format)
    '''
    tables = soup.find_all('table') # search tab <table>, return soup (list)
    table = tables[0] #using the 1st table
    rows = table.find_all('tr') #find all the rows in the table 
    data = []
    
    for row in rows[1:]:
        data_row = [] # create a new row for data table
        tds = row.find_all('td')
        # 1st element only has text
        data_row.append(tds[0].text) #find brings 1st 
        # 2nd element only has text
        data_row.append(tds[1].find('a').text)
        # grad link in 2nd element 
        data_row.append(tds[1].find('a')['href'])
        # 3rd elem & link
        data_row.append(tds[2].find('a').text)
        data_row.append(tds[2].find('a')['href'])
        # 4th-6th only has text
        data_row.append(tds[3].text)
        data_row.append(tds[4].text)
        data_row.append(tds[5].text)
        # print * = unpack
    #    print(*data_row, sep='\n')
        data.append(data_row) # add rows to data table 
    return data

## repeat process_page for all pages

In [43]:
from os.path import join
print(join(BASE_URL,'//3')) #did not work properly 

//3


In [44]:
from os.path import join #auto correctly join urls 
import time

the_movies_data=[]
for i in range(1, MAX_PAGE+1, 100): #stepsize = 100
#   print(i)  ## check pages are corrrect 
    resp = requests.get(join(BASE_URL, str(i)))
    soup = BeautifulSoup(resp.text, 'html.parser')
    the_movies_data += process_page(soup) #concat, not append
    # the_movies_data.extend(process_page(soup))
    # time.sleep(3) # put to sleep if takes too long? 

print(len(the_movies_data))    

5785


In [46]:
# fix headers
headers.insert(2,'Resease_Date_URL')
headers.insert(4,'Movie_URL')

## SQL

In [47]:
import sqlite3

### grab table in memory

In [71]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [72]:
headers[0] = 'Idx' # Index is protected by SQL

In [73]:
column_names = ','.join([header.replace(' ','_') + ' varchar\n' for header in headers])
print(column_names)

Idx varchar
,Release_Date varchar
,Resease_Date_URL varchar
,Movie varchar
,Movie_URL varchar
,Production_Budget varchar
,Domestic_Gross varchar
,Worldwide_Gross varchar



In [74]:
sql = """
CREATE TABLE movies({col_names})
"""
cur.execute(sql.format(col_names = column_names))

In [75]:
cur.execute("select * from movies;")
cur.fetchall()

[]

In [76]:
insert_sql = """
INSERT INTO movies VALUES(?,?,?,?,?,?,?,?)
"""
for d in the_movies_data[0:]:
    cur.execute(insert_sql, the_movies_data[0])

In [79]:
cur.execute("select * from movies;")
cur.fetchone()

('1',
 'Dec 18, 2009',
 '/box-office-chart/daily/2009/12/18',
 'Avatar',
 '/movie/Avatar#tab=summary',
 '$425,000,000',
 '$760,507,625',
 '$2,776,345,279')

In [81]:
resp.request.headers

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [ ]:
# request with changing User-Agent (if forbidden request)

In [ ]:
pd.read_html("")

## concat two links:
**os.path.join(path, *path)** # e.g., automatically get rid of extra '/'


## import time, time.sleep